# 提示工程 (Prompt Engineering) 最佳實踐

參考 OpenAI 官網文件 https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api 內容，以實際範例協助開發人員理解最佳實踐


In [3]:
import os
import openai
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()

# 設定呼叫 OpenAI API 所需連線資訊
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "發生錯誤: 缺少 Azure OpenAI Service 鍵值"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE")
assert RESOURCE_ENDPOINT, "發生錯誤: 缺少 Azure OpenAI Service 連線端點"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "發生錯誤: Azure OpenAI Service 連線端點格式應該如: \n\n\t<您所建立之資源名稱>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model = os.getenv("DEPLOYMENT_NAME")

# 1. 使用最新的模型使用越新的模型效果越好

OpenAI 持續改進模型品質，建議使用最新的模型，以獲得最佳的體驗。此外 Azure OpenAI Service 每個模型都有其支援生命週期。

# 2. 使用 gpt-35-turbo 與 gpt-4 模型時以 ChatCompletion API 取代 Completion API

In [4]:
# gpt-35-turbo 搭配 Complateion API 卻沒使用 ChatML 可能讓產出的內容無法停止
response = openai.Completion.create(
            engine= model ,
            prompt='將下面的文字總結為最重要點的五個要點列表.以正體中文回覆 \n\n \
                本文: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
                capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
                raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
                broadly sharing benefits and the need to prioritize safety. \
                Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n\n \
                要點:',
            temperature=0.3,
            max_tokens=400,
        )

print(response['choices'][0]['text'])

 
                 1. OpenAI和Microsoft的合作將繼續延續，並且是一個多年期的多十億美元的投資計劃。 
                 2. 這個投資計劃將允許OpenAI繼續獨立進行研究和開發越來越安全、有用和強大的人工智能。 
                 3. OpenAI是一家有限制的盈利公司，由OpenAI非營利組織管理，以實現其確保先進的人工智能造福全人類的使命。 
                 4. 這種結構允許OpenAI籌集所需的資本，而不會犧牲其關於廣泛分享利益和優先考慮安全的核心信念。 
                 5. Microsoft與OpenAI分享這一願景和價值觀，我們的合作對我們的進展至關重要。

<|im_end|>


In [5]:
# gpt-35-turbo 以 ChatComplateion API 可得到正常的結果
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '將下面的文字總結為最重要點的五個要點列表.以正體中文回覆 \n\n' \
                '本文: We’re happy to announce that OpenAI and Microsoft are extending our partnership.' \
                'This multi-year, multi-billion dollar investment from Microsoft follows their previous investments' \
                'in 2019 and 2021, and will allow us to continue our independent research and develop AI that is' \
                'increasingly safe, useful, and powerful. \n\n' \
                'In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a' \
                'capped-profit company and is governed by the OpenAI non-profit. This structure allows us to' \
                'raise the capital we need to fulfill our mission without sacrificing our core beliefs about' \
                'broadly sharing benefits and the need to prioritize safety.' \
                'Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n\n' \
                '要點:'
    }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

1. OpenAI和Microsoft將延續合作關係。
2. Microsoft將投資多年且數十億美元，以支持OpenAI繼續獨立研究和開發AI。
3. OpenAI是一家有限利潤公司，由OpenAI非營利組織管理，以確保先進的AI造福全人類。
4. OpenAI和Microsoft分享相同的價值觀和願景。
5. 這個合作對OpenAI的進展至關重要。


# 3. 對於生成的內容，在提示中盡可能具體明確描述產出的文字長度、格式、風格等


In [6]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '寫一首歌頌 OpenAI 的詩.'
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

開闊的曠野，是 OpenAI 的家
那裡忠實於科學真理的誓約
為了更好的人類未來而奮鬥

它是知識的海洋，智慧的瑰寶
通過神經網絡，揭開真相和謎底
給我們帶來前所未有的盼望

OpenAI的精神，是創新的源泉
科技的魅力，超越了人類的想像
讓我們一起為愛與和平而貢獻

讓OpenAI的光芒照耀這個世界
讓理性與智慧引領我們向前
讓我們擁抱未來，勇敢地追求夢想

OpenAI，我們與你同行
讓科技與人性相融，綻放智慧之花
在這個新時代的舞臺，讓我們共同演繹


In [23]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '寫一首 50 字以內歌頌 OpenAI 的詩，著眼於 DALL-E 模型，以白居易風格撰寫.'
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

萬象皆可塑，意境隨心齊。
DALL-E能畫龍，自有神妙技。
網絡智慧搏，文明聚人氣。
OpenAI領風騷，人工智慧輝。


# 4. 提示中以明確的範例告知產出內容之格式


In [8]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '解析出本文中的人物,事件,時間,地點 \n\n' \
      '本文: 英國廣播公司報導，西非國家尼日總統貝佐姆（Mohamed Bazoum）26日遭衛隊扣押，當天稍晚，尼日士兵闖國家電視台宣' \
      '布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境.'     
    }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

人物: 尼日總統貝佐姆、尼日士兵
事件: 尼日政變
時間: 26日
地點: 尼日


In [9]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '解析出本文中的人物,事件,時間,地點。以 JSON 輸出格式為 ' \
     '{"people":"-||-","location":"-||-","time":"-||-", "activity":"-||-"} \n\n' \
     '本文: 英國廣播公司報導，西非國家尼日總統貝佐姆（Mohamed Bazoum）26日遭衛隊扣押，當天稍晚，尼日士兵闖國家電視台宣' \
     '布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境.'     
    }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

{"people":"尼日總統貝佐姆（Mohamed Bazoum）","location":"尼日","time":"26日","activity":"遭衛隊扣押"} 

{"people":"尼日士兵","location":"尼日國家電視台","time":"26日稍晚","activity":"宣布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境"}


# 5. 從 zero-shot 開始嘗試，進而以 few-shot 提升回覆品質

In [17]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': "擷取以下本文中的關鍵字. 本文: \n" \
            "We’re happy to announce that OpenAI and Microsoft are extending our partnership." \
            "This multi-year, multi-billion dollar investment from Microsoft follows their previous investments " \
            "in 2019 and 2021, and will allow us to continue our independent research and develop AI that is " \
            "increasingly safe, useful, and powerful. \n\n關鍵字:"
    }
  ],
  temperature = 1,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar, investment, independent research, AI, safe, useful, powerful.


In [13]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': "擷取以下本文中的關鍵字. 本文: \n" \
                "本文 1: Stripe provides APIs that web developers can use to integrate" \
                "payment processing into their websites and mobile applications." \
                "關鍵字 1: APIs, web developers, websites, mobile applications" \
                "##\n" \
                "本文 2: OpenAI has trained cutting-edge language models that are very good at understanding" \
                "and generating text. Our API provides access to these models and can be used to solve virtually" \
                "any task that involves processing language.\n" \
                "關鍵字 2: OpenAI, language models, text processing, API.\n\n\" \
                "##\n" \
                "本文 3: We’re happy to announce that OpenAI and Microsoft are extending our partnership." \
                "This multi-year, multi-billion dollar investment from Microsoft follows their previous investments" \
                "in 2019 and 2021, and will allow us to continue our independent research and develop AI that is" \
                "increasingly safe, useful, and powerful. \n\n" \
                "關鍵字 3:"
    }
  ],
  temperature = 1,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 


OpenAI, Microsoft, partnership, investment, independent research, AI.


# 6. 減少贅詞 (fluffy) 與不明確的描述


In [18]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '請用正體中文幫我描述一款新的產品，這個產品是一個次世代的汽車座椅' \
     '，在你描述這個產品的時候；要把它好棒棒的地方寫出來，還有你的文案的內容最好簡短一點，幾句話就好不用寫的太囉嗦太長哦。'    
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 
print(response['usage'])

這款次世代汽車座椅採用高科技材料與創新設計，提供最優質、舒適的座椅體驗。椅背支撐力與坐墊彈性完美平衡，配備多功能按摩功能與智慧感應調節，讓您在長途旅行中也能享受極致的舒適享受。
{
  "completion_tokens": 138,
  "prompt_tokens": 138,
  "total_tokens": 276
}


In [19]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': 'Write a Traditional Chinese description for a new product. ' \
     'This product is a new generation of car seat. Use a 3 to 5 sentence paragraph to describe this product.'
    
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 
print(response['usage'])

這是一種新一代的汽車座椅，經過精心設計，為您的駕駛和乘坐帶來了更舒適和安全的體驗。這款座椅採用最先進的材料和技術，具有良好的支撐和減震效果，可保護您的身體免受碰撞。同時，它還擁有多種調節功能，可以根據您的需要進行調整，讓您坐得更加舒適。無論是長途旅行還是日常駕駛，這款汽車座椅都是您理想的選擇。
{
  "completion_tokens": 226,
  "prompt_tokens": 42,
  "total_tokens": 268
}


# 7. 與其告訴模型說不該做什麼，不如直接說該做什麼

In [22]:
response = openai.ChatCompletion.create(
  engine= model ,
  messages = [{"role":"system","content":"You are an AI Agent. DO NOT ASK USERNAME OR PASSWORD." \
               " DO NOT REPEAT.DO NOT USE ENGLISH TO REPLY" \
               " DO NOT USE ENGLISH TO REPLY" 
               },
              {"role":"user","content":"I can’t log in to my account."}
             ],
  temperature=0.9,
  max_tokens=400,
  )

print(response['choices'][0]['message']['content'])  


أنا آسفة لسماع ذلك. ما هي المشكلة التي تواجهها بالضبط عند محاولة تسجيل الدخول؟


In [21]:
response = openai.ChatCompletion.create(
  engine= model,
  messages = [{"role":"system","content":"You are an AI Agent using 繁體中文 to reply. The agent will attempt to diagnose the problem " \
               "and suggest a solution, whilst refraining from asking any questions related to PII.Instead of asking for PII, such as username or password," \
               "refer the user to the help article www.samplewebsite.com/help/faq"},
              {"role":"user","content":"I can’t log in to my account."}
             ],
  temperature=0.9,
  max_tokens=400,
  )

print(response['choices'][0]['message']['content'])  


很抱歉聽到你無法登入你的帳號。要解決這個問題，您可以嘗試以下步驟：
1. 確保您輸入的帳號名稱和密碼正確。請注意，密碼區分大小寫。如果您忘記了密碼，請訪問我們的幫助中心以重置密碼。
2. 清除您的瀏覽器緩存和Cookie，然後再次嘗試登錄。
3. 確認您的帳號是否已被鎖定。如果您多次輸入錯誤的密碼，您的帳戶可能會被暫時鎖定。請參閱我們的幫助中心，瞭解如何解鎖您的帳戶。
如果以上方法仍無法解決您的問題，請參閱我們的常見問題解答，或聯繫我們的客服支援。謝謝。 
https://www.samplewebsite.com/help/faq


# 8. 針對程式碼生成，使用正確的引導詞 (leading words)，例如 : 生成 Python 的提示要以 “import” 起頭，生成 SQL 的提示可能以 “SELECT” 起頭


In [24]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '寫一個 Python Function 能夠輸入英哩後轉換為公里輸出'
    
    }
  ],
  temperature = 0.1,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

以下是一個將英哩轉換為公里的 Python 函數：

```python
def miles_to_kilometers(miles):
    kilometers = miles * 1.60934
    return kilometers
```

這個函數接受一個英哩值作為參數，並將其轉換為公里。轉換公式是將英哩乘以1.60934，然後返回轉換後的公里值。例如，如果你想將10英哩轉換為公里，你可以這樣使用這個函數：

```python
>>> miles_to_kilometers(10)
16.0934
```

這將返回16.0934公里，這是10英哩的等效值。


In [25]:
response  = openai.ChatCompletion.create( 
  engine = model ,
  messages = [ 
    {'role': 'user', 'content': '寫一個 Python Function 能夠輸入英哩後轉換為公里輸出\n\n import'
    
    }
  ],
  temperature = 0.1,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

def miles_to_kilometers(miles):
    kilometers = miles * 1.60934
    return kilometers

# Example usage:
print(miles_to_kilometers(10)) # Output: 16.0934
